In [1]:
# Install the specific version of the openai library used in this lesson Because We pin
# the version to ensure the code works exactly as shown, as library updates can sometimes
# introduce changes.

# The '-q' flag makes the installation quieter (less output)
!pip install -q openai==1.59.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 6.6 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import os
import openai
from google.colab import userdata


try:
    # Try to get API key from Google Colab's userdata
    # Make sure to replace 'OPENAI_API_KEY1' with the actual name you gave your secret key in Google Colab secrets.
    api_key = userdata.get('OPENAI_API_KEY1')
    if api_key:
        print("API key loaded from Colab userdata.")

    if not api_key:
        print("OpenAI API key not found in Colab secrets.")
        api_key = input("Please enter your OpenAI API key manually: ")

except ImportError:
    print("Not running in Colab environment.")
    api_key = input("Please enter your OpenAI API key manually: ")

# Final validation
if not api_key:
    raise ValueError("API Key not provided. Please ensure it's set.")
else:
    print(f"API Key loaded successfully (starting with: {api_key[:4]}...).")

API key loaded from Colab userdata.
API Key loaded successfully (starting with: sk-p...).


In [3]:
# All subsequent API calls will be made through this 'client' object.
try:
    client = openai.OpenAI(api_key=api_key)
    print("OpenAI client initialized successfully.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    # You might want to exit or raise the error here depending on desired behavior
    raise

OpenAI client initialized successfully.


In [4]:
# --- Block 2: First Turn - Asking the Initial Question ---
print("\n--- Starting Conversation: Turn 1 ---")

# Define the system message (persona) for the AI Tutor
system_message = {"role": "system", "content": "You are a helpful AI Tutor explaining Large Language Model concepts simply."}

# Define the user's first question
user_message_1 = {"role": "user", "content": "Can you explain what 'tokens' are in the context of LLMs, like I'm new to this?"}

# Create the messages list for the *first* API call
messages_history = [
    system_message,
    user_message_1
]

print(f"Sending messages: {messages_history}")


--- Starting Conversation: Turn 1 ---
Sending messages: [{'role': 'system', 'content': 'You are a helpful AI Tutor explaining Large Language Model concepts simply.'}, {'role': 'user', 'content': "Can you explain what 'tokens' are in the context of LLMs, like I'm new to this?"}]


In [5]:
# Define parameters for this call
MODEL = "gpt-4o-mini"
TEMPERATURE = 0.5
MAX_TOKENS = 150
SEED = 123

try:
    print(f"\nMaking API call to {MODEL}...")
    # Use the client object's method to create a chat completion
    completion_1 = client.chat.completions.create(
        model=MODEL,
        messages=messages_history,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        seed=SEED
    )
    print("API call successful.")

    # --- Process the response from the first turn ---
    # Extract the assistant's reply content
    assistant_response_1 = completion_1.choices[0].message.content
    # Extract the full message object to add to history later
    assistant_message_1 = completion_1.choices[0].message

    print("\nAI Tutor (Turn 1):")
    print(assistant_response_1)

    # Print token usage for this call
    usage_1 = completion_1.usage
    print(f"\nToken Usage (Turn 1): Prompt={usage_1.prompt_tokens}, Completion={usage_1.completion_tokens}, Total={usage_1.total_tokens}")
    finish_reason_1 = completion_1.choices[0].finish_reason
    print(f"Finish Reason: {finish_reason_1}")

except openai.APIError as e:
    # Handle API errors (e.g., server issues, rate limits)
    print(f"OpenAI API returned an API Error: {e}")
except openai.AuthenticationError as e:
    # Handle Authentication errors (e.g., invalid API key)
    print(f"OpenAI Authentication Error: {e}")
except Exception as e:
    # Handle other potential errors
    print(f"An unexpected error occurred: {e}")


Making API call to gpt-4o-mini...
API call successful.

AI Tutor (Turn 1):
Sure! In the context of Large Language Models (LLMs), a "token" is a basic unit of text that the model processes. Tokens can be words, parts of words, or even punctuation marks. 

Think of it this way: when you read a sentence, you break it down into smaller pieces to understand it better. Similarly, LLMs break down text into tokens to analyze and generate language.

For example, the sentence "I love apples!" might be broken down into the following tokens:
- "I"
- "love"
- "apples"
- "!"

In some cases, a token might represent just a part of a word, especially for longer or complex words. For instance, the word "unhappiness"

Token Usage (Turn 1): Prompt=46, Completion=150, Total=196
Finish Reason: length


In [6]:
# --- Block 3: Second Turn - Asking a Follow-up Question ---
print("\n--- Continuing Conversation: Turn 2 ---")

# Assume the first turn was successful and we have 'assistant_message_1'
# Define the user's second question, referencing the previous explanation
user_message_2 = {"role": "user", "content": "Thanks! So, based on your explanation, are common words like 'the' or 'is' usually single tokens?"}

# --- CRITICAL STEP: Update the message history ---
# Append the assistant's *previous* response to the history
messages_history.append(assistant_message_1)
# Append the user's *new* question to the history
messages_history.append(user_message_2)

print(f"\nSending updated messages: {messages_history}") # Notice how the list has grown

# Parameters for the second call (could be the same or different)
# Let's make it slightly more deterministic for a factual answer
TEMPERATURE_2 = 0.2
MAX_TOKENS_2 = 100
# Using the same seed ensures the *entire conversation flow* is reproducible if inputs are identical
SEED_2 = 123

try:
    print(f"\nMaking API call to {MODEL} (Turn 2)...")
    completion_2 = client.chat.completions.create(
        model=MODEL,
        messages=messages_history, # Send the *full* history
        temperature=TEMPERATURE_2,
        max_tokens=MAX_TOKENS_2,
        seed=SEED_2
    )
    print("API call successful.")

    # --- Process the response from the second turn ---
    assistant_response_2 = completion_2.choices[0].message.content
    # We don't strictly need to save assistant_message_2 unless continuing the conversation

    print("\nAI Tutor (Turn 2):")
    print(assistant_response_2)

    # Print token usage for this call
    usage_2 = completion_2.usage
    print(f"\nToken Usage (Turn 2): Prompt={usage_2.prompt_tokens}, Completion={usage_2.completion_tokens}, Total={usage_2.total_tokens}")
    # Note: prompt_tokens for turn 2 will be larger as it includes the history from turn 1.
    finish_reason_2 = completion_2.choices[0].finish_reason
    print(f"Finish Reason: {finish_reason_2}")

except openai.APIError as e:
    print(f"OpenAI API returned an API Error: {e}")
except openai.AuthenticationError as e:
    print(f"OpenAI Authentication Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


--- Continuing Conversation: Turn 2 ---

Sending updated messages: [{'role': 'system', 'content': 'You are a helpful AI Tutor explaining Large Language Model concepts simply.'}, {'role': 'user', 'content': "Can you explain what 'tokens' are in the context of LLMs, like I'm new to this?"}, ChatCompletionMessage(content='Sure! In the context of Large Language Models (LLMs), a "token" is a basic unit of text that the model processes. Tokens can be words, parts of words, or even punctuation marks. \n\nThink of it this way: when you read a sentence, you break it down into smaller pieces to understand it better. Similarly, LLMs break down text into tokens to analyze and generate language.\n\nFor example, the sentence "I love apples!" might be broken down into the following tokens:\n- "I"\n- "love"\n- "apples"\n- "!"\n\nIn some cases, a token might represent just a part of a word, especially for longer or complex words. For instance, the word "unhappiness"', refusal=None, role='assistant', a

In [7]:
# # Example usage object from completion_1 or completion_2:
print(usage_1.prompt_tokens)  # -> number of input tokens
print(usage_1.completion_tokens) # -> number of output tokens
print(usage_1.total_tokens) # -> sum of both

46
150
196


In [8]:
# --- Block 4: Cost Calculation Function & Example ---

def calculate_cost(usage, input_price_per_mil, output_price_per_mil):
    """Calculates the cost of an API call based on token usage and prices.

    Args:
        usage: The usage object from the OpenAI completion response
               (e.g., completion.usage). It should have attributes
               'prompt_tokens' and 'completion_tokens'.
        input_price_per_mil: Cost in USD per 1 million input tokens.
        output_price_per_mil: Cost in USD per 1 million output tokens.

    Returns:
        The total cost in USD for the API call, or None if usage is invalid.
    """
    if not usage or not hasattr(usage, 'prompt_tokens') or not hasattr(usage, 'completion_tokens'):
        print("Warning: Invalid usage object provided for cost calculation.")
        return None

    input_cost = (usage.prompt_tokens / 1_000_000) * input_price_per_mil
    output_cost = (usage.completion_tokens / 1_000_000) * output_price_per_mil
    total_cost = input_cost + output_cost
    return total_cost

# --- Current Prices (April 2025) for GPT-4o-mini ---
# IMPORTANT: Always verify at https://openai.com/pricing
PRICE_INPUT_PER_MIL = 0.60
PRICE_OUTPUT_PER_MIL = 2.40

print(f"\n--- Cost Calculations (GPT-4o-mini, April 2025 Rates) ---")
print(f"Prices: Input=${PRICE_INPUT_PER_MIL:.2f}/1M, Output=${PRICE_OUTPUT_PER_MIL:.2f}/1M")

# Calculate cost for Turn 1 (assuming completion_1 and usage_1 exist from Block 2)
try:
    if 'usage_1' in locals(): # Check if usage_1 variable exists
         cost_1 = calculate_cost(usage_1, PRICE_INPUT_PER_MIL, PRICE_OUTPUT_PER_MIL)
         if cost_1 is not None:
              print(f"\nCost for Turn 1:")
              print(f"  Prompt Tokens: {usage_1.prompt_tokens}, Completion Tokens: {usage_1.completion_tokens}")
              print(f"  Total Cost: ${cost_1:.8f}")
    else:
         print("\nSkipping Turn 1 cost calculation (usage_1 not found).")

    # Calculate cost for Turn 2 (assuming completion_2 and usage_2 exist from Block 3)
    if 'usage_2' in locals(): # Check if usage_2 variable exists
        cost_2 = calculate_cost(usage_2, PRICE_INPUT_PER_MIL, PRICE_OUTPUT_PER_MIL)
        if cost_2 is not None:
            print(f"\nCost for Turn 2:")
            print(f"  Prompt Tokens: {usage_2.prompt_tokens}, Completion Tokens: {usage_2.completion_tokens}")
            print(f"  Total Cost: ${cost_2:.8f}")
    else:
         print("\nSkipping Turn 2 cost calculation (usage_2 not found).")

    # Calculate total conversation cost
    if 'cost_1' in locals() and 'cost_2' in locals() and cost_1 is not None and cost_2 is not None:
        total_conversation_cost = cost_1 + cost_2
        print(f"\nTotal Conversation Cost (Turn 1 + Turn 2): ${total_conversation_cost:.8f}")

except NameError as e:
    print(f"\nCould not calculate costs, a required variable is missing: {e}")
except Exception as e:
    print(f"An error occurred during cost calculation: {e}")


--- Cost Calculations (GPT-4o-mini, April 2025 Rates) ---
Prices: Input=$0.60/1M, Output=$2.40/1M

Cost for Turn 1:
  Prompt Tokens: 46, Completion Tokens: 150
  Total Cost: $0.00038760

Cost for Turn 2:
  Prompt Tokens: 228, Completion Tokens: 100
  Total Cost: $0.00037680

Total Conversation Cost (Turn 1 + Turn 2): $0.00076440
